# Best Actor - Mobile Game about Facial Expression based on Convolutional Neural Network

## Abstract

TODO

## Terminology

TODO

## UI & UX Design

TODO

## Technical Design

TODO

## Code Implementation

### Part 1: Computer Vision

#### **Preprocess Data**

First and foremost, we need to preprocess the dataset. Here I use **FER2013** dataset for the facial expression recognition, which contains 7 categories in total, including (1) Angry, (2) Disgust, (3) Fear, (4) Happy, (5) Neutral, (6) Sad and (7) Surprise. The examples of FER2013 dataset are illustrated as follows.

![Facial Expressions](https://drive.google.com/uc?id=11VQZQZLD0_GwkcR4vUWTNtBdbbZS0Dh7)

The dataset has the following advantages:
- It contains dataset with various ages
![age](https://drive.google.com/uc?id=1mUcafNru8bb6pn-6xFkW74qIQVpI9Jyh)
- It contains dataset with different ethics
![ethic](https://drive.google.com/uc?id=1bZf1BR-fVZvnHmlgT-ZDbJKRQULxfOJ7)
- It contains dataset with accessories
![accessory](https://drive.google.com/uc?id=1xmEbRDzKBeDRpGa63hGH-kIBOGMXMJJp)

However, the dataset also has the following disadvantages:
- Some images have watermark, which might have potential copyright issue
![watermark](https://drive.google.com/uc?id=142nfQZ3a1hMdFhQIr2-V3wmxBEvYFDhz)
- Some images are cartoon characters
![cartoon](https://drive.google.com/uc?id=1UTUDU5ac8aTcfR9O7WTqDUkGgj-1RfkJ)
- Some images have inconsistent facial expressions and labels. For example, the following facial expressions are marked as "Happy" in FER2013 dataset, which are apparently not "Happy" facial expressions
![inconsistent](https://drive.google.com/uc?id=1Gu0SNL7B3uoQ7TEmv1LRe3-97NCET9iX)
- Some images are broken and have no facial expressions
![non-people](https://drive.google.com/uc?id=1sbeWG41IOot29GOY6iCAwRQPG71-Euhk)
- Some images have multiple faces
![many](https://drive.google.com/uc?id=1l8PPTBHb7BPS7LukOAdVV0yM1ZaBSAlQ)

Therefore, I preprocessed the dataset by removing ineligible images. Then, I created the preprocessed "train.csv" with the following function.
```python
from PIL import Image
import os
import imghdr
import numpy
import csv

# Constants
TRAINING_DATA_DIRECTORY = 'TrainingDataSelected/'
TRAIN_CSV_FILE = 'train.csv'
TRAIN_CSV_WRITE_MODE = 'w'
TRAIN_DATA_IMAGE_TYPE = 'jpeg'

FACIAL_EXPRESSION_LABELS = {
    'Happy': '0',
    'Sad': '1',
    'Surprise': '2'
}

class Preprocessor:

    def preprocess_data(self):
        """Preprocess facial expression dataset.
        This function will save training data to train.csv in the directory.
        train.csv contains two columns:
        - Facial expression label
        - Numpy array of facial expression image file (48 x 48)
        """
        with open(TRAIN_CSV_FILE, mode=TRAIN_CSV_WRITE_MODE) as train:

            # Create train writer
            train_writer = csv.writer(train)

            # Iterate through training image data
            with os.scandir(TRAINING_DATA_DIRECTORY) as entries:

                for entry in entries:
                    path = entry.path
                    file_type = imghdr.what(path)

                    if file_type == TRAIN_DATA_IMAGE_TYPE:

                        # Label
                        name = entry.name
                        facial_expression_name = name.split('_')[0]
                        label = FACIAL_EXPRESSION_LABELS[facial_expression_name]

                        # Image file
                        image_file = Image.open(path)
                        image = numpy.array(image_file)

                        # Write to CSV file
                        row = [label, image]
                        train_writer.writerow(row)
```

#### **Load Data**

TODO

#### **Tune Hyperparameter**

TODO

#### **Train Data**

TODO

#### **Evaluate Model**

TODO

#### **Export Model**

TODO

#### **Convert CoreML Model**

TODO

### Part 2: Mobile Game

#### **Crop ROI with OpenCV**

TODO

#### **Predict using CoreML Model**

TODO

## Video Demonstration

TODO

## Conclusion

TODO

## Reference

TODO